# Jupyter Notebook for inserting coordinates in a GPR file

### Task:

Insert a new set of (X,Y) coordinates, analytically predefined as a function of file progressive reference number, into the header of each single trace. This approach works for the data recorded in "ScienzaU4" during the course because we know approximately the (X,Y)-grid on which the antenna has been moved along. In other cases, (X,Y)-coordinates should be read separately from a file or the function modified.

Old files are read from the directory: __data-OLD/__ or downloaded from INGV FTP site

New files are saved in the directory:  __data-NEW-1-constant-length/__

We also compute and alternative set of coordinates, based on the assumption that all traces are spaced 0.96 cm along X coordinate

New files are saved in the directory:  __data-NEW-2-constant-spacing/__



### Workflow:

(1) Define coordinates as a function of file reference number (ONLY for scienzaU4 data)

(2) Download data 

(3) Loop on all data files:

(3.1) Read one data 

(3.2) Compute X coordinates for the traces in the data file

(3.3) Insert the X and Y coordinates and write the new data file

(3.4) Read new data file header and double-check coordinates


###  Import libraries 

In [1]:
# Per la gestione dei file
import sys
import os
import ftplib

# Per il calcolo numerico
import numpy as np
from scipy import signal

# Per la presentazione grafica dei risultati
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches


# Per l'analisi dei dati SEGY
import obspy
from obspy.io.segy.core import _read_segy

# Per le parti interattive del Notebook
from ipywidgets import interact, Layout
import ipywidgets as widgets

#Per visualizzare immagini
from IPython.display import Image
import matplotlib.image as image 

import struct

from read_SEGY_PRISM_header import npa_read_coord_in_SEGY_PRISM_header


### (1) Define (X,Y) coordinates   --- only ODD traces

In [3]:
#
# Here:
#
# (1) Y-coordinates are taken from Giorgia's file
# (2) X_coord depends on segment and are taken from the graph the students did in the field: 
#
#                                 "FULL" - segments are all starting at 0 m and ending af 12 m.
#                                 "FIRST-HALF" - segments start at 0 m and end about 6 m
#                                 "SECON-HALF" - segments start at different points and end at 12 m
#
#


#KIND_of_SEGMENT = "FULL"                 # "FULL",  "FIRST-HALF", "SECOND-HALF"
KIND_of_SEGMENT = "SECOND-HALF"           # "FULL",  "FIRST-HALF", "SECOND-HALF"
CHANNEL = "2"                            # 1- 10309.3Hz  2- 20833.3Hz


if KIND_of_SEGMENT == "FULL":

    nseg=17
    CHANNEL_list = ['01','03','05','15','17','19','23','25','27','29','31','33','35','37','39','41','43']
    Y_coord = [0.25,0.75,1.25,3.74,4.24,4.74,5.74,6.24,6.74,7.24,7.74,8.23,8.74,9.24,9.73,10.23,10.7]
    start = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    length = [12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0]
    
if KIND_of_SEGMENT == "FIRST-HALF":

    nseg=5
    CHANNEL_list = ['07','09','11','13','21']
    Y_coord = [1.75,2.25,2.75,3.25,5.24]
    start = [0,0,0,0,0]
    length = [6.5,6.0,6.5,6.5,10.0]
    
if KIND_of_SEGMENT == "SECOND-HALF":

    nseg=5
    CHANNEL_list = ['07.2','09.2','11.2','13.2','21.2']
    Y_coord = [1.75,2.25,2.75,3.25,5.24]
    start = [8.0,9.0,9.5,9.5,10.5]
    length = [4.0,3.0,2.5,2.5,1.5]
    

### (2) Download data

In [4]:

for STAT in CHANNEL_list:
    FTP_host = "ftp.ingv.it"
    ftp_path = "/pub/nicola.piana/UNIMIB-lectures/"
    INDIR1='data-GPR/'
    DASHfile='scienzaU4_' + STAT + '_CH_2.sgy'

    filename = './data-OLD/' + DASHfile
    data_stream = []


    if os.path.exists( filename ):
            print('GPR data already donwloaded ... skipping FTP phase')
    else:
        DASH_file_ORIG= INDIR1 + DASHfile    
        ftp = ftplib.FTP(FTP_host) 
        ftp.login() 
        ftp.cwd(ftp_path)
        print('Dowloading file:', DASH_file_ORIG, ' in file:', filename)
        ftp.retrbinary("RETR " + DASH_file_ORIG, open(filename, 'wb').write)
        print('Dowloading file:', DASH_file_ORIG, ' completed ...' )
        ftp.quit()





GPR data already donwloaded ... skipping FTP phase
GPR data already donwloaded ... skipping FTP phase
GPR data already donwloaded ... skipping FTP phase
GPR data already donwloaded ... skipping FTP phase
GPR data already donwloaded ... skipping FTP phase


### Define function for inserting coordinates

In [5]:
x = []
y = []

def insert_coords(GPRfile_in,ntraces,x,y,GPRfile_out):
    
    with open(GPRfile_in,"rb") as f:
        data = bytearray(f.read())
        
    il=0
    
    while il < ntraces:
        h_start=3600+il*(240+1024*2)
        x_0 = x[il]
        y_0 = y[il]
        data[h_start+182:h_start+190] = struct.pack('d',x_0)  
        data[h_start+190:h_start+198] = struct.pack('d',y_0) 
        il += 1
        
    with open(GPRfile_out,'wb') as f:
        f.write(data)


### (3) Loop on all data files:

In [6]:


istat=0
for STAT in CHANNEL_list:

    GPRfile_old = 'data-OLD/scienzaU4_' + STAT + '_CH_' + CHANNEL + '.sgy'
    GPRfile_new = 'data-NEW-1-constant-length/scienzaU4_' + STAT + '_CH_' + CHANNEL + '.sgy'
    
    print('\n -----------------\n  READING FILE:', GPRfile_old, '\n  WRITING FILE:', GPRfile_new)

    
    # (3.1) Read one data file
    
    data_stream = []
    data_stream.append(_read_segy(GPRfile_old))
    #print(data_stream[0])
    ntraces = len(data_stream[0])
    print('  NTRACES in FILE:', ntraces)
    
    # (3.2) Compute X coordinates for the traces in the data file

    x = []
    y = []
    il=0
    while il < ntraces:
        y_0 = Y_coord[istat]
        y.append(y_0)
        x_0 = start[istat] + il * (length[istat]/(ntraces-1))
        x.append(x_0)
        il+=1
        
    # (3.3) Insert the X and Y coordinates and write the new data file
    
    insert_coords(GPRfile_old,ntraces,x,y,GPRfile_new)
    
    # (3.4) Read new data file header and double-check coordinates
       
    gpr_lon, gpr_lat, gpr_mark = npa_read_coord_in_SEGY_PRISM_header(GPRfile_new,ntraces)
        
    istat+=1


 -----------------
  READING FILE: data-OLD/scienzaU4_07.2_CH_2.sgy 
  WRITING FILE: data-NEW-1-constant-length/scienzaU4_07.2_CH_2.sgy
  NTRACES in FILE: 382

Encoding Byte Type:  3
Delta (in picos):  49
Nsamples:  1024


     0  LON/LAT:       8.00000000      1.75000000  MARK:    255
     0  LON/LAT:       8.00000000      1.75000000  MARK:    255
   381  LON/LAT:      12.00000000      1.75000000  MARK:      1

 -----------------
  READING FILE: data-OLD/scienzaU4_09.2_CH_2.sgy 
  WRITING FILE: data-NEW-1-constant-length/scienzaU4_09.2_CH_2.sgy
  NTRACES in FILE: 303

Encoding Byte Type:  3
Delta (in picos):  49
Nsamples:  1024


     0  LON/LAT:       9.00000000      2.25000000  MARK:    255
     0  LON/LAT:       9.00000000      2.25000000  MARK:    255
   302  LON/LAT:      12.00000000      2.25000000  MARK:      1

 -----------------
  READING FILE: data-OLD/scienzaU4_11.2_CH_2.sgy 
  WRITING FILE: data-NEW-1-constant-length/scienzaU4_11.2_CH_2.sgy
  NTRACES in FILE: 284

Encodin

### (3) Loop on all data files -- Alternative X coordinate computation

In [11]:


istat=0
for STAT in CHANNEL_list:

    GPRfile_old = 'data-OLD/scienzaU4_' + STAT + '_CH_' + CHANNEL + '.sgy'
    GPRfile_new = 'data-NEW-2-constant-spacing/scienzaU4_' + STAT + '_CH_' + CHANNEL + '.sgy'
    
    print('\n -----------------\n  READING FILE:', GPRfile_old, '\n  WRITING FILE:', GPRfile_new)

    
    # (3.1) Read one data file
    
    data_stream = []
    data_stream.append(_read_segy(GPRfile_old))
    #print(data_stream[0])
    ntraces = len(data_stream[0])
    print('  NTRACES in FILE:', ntraces)
    
    # (3.2) Compute X coordinates for the traces in the data file

    x = []
    y = []
    il=0
    while il < ntraces:
        y_0 = Y_coord[istat]
        y.append(y_0)
        if KIND_of_SEGMENT == "SECOND-HALF":
            x_0 = start[istat] + length[istat] - (ntraces-1-il) * 0.0096
        else:
            x_0 = start[istat] + il * 0.0096
        x.append(x_0)
        il+=1
        
    # (3.3) Insert the X and Y coordinates and write the new data file
    
    insert_coords(GPRfile_old,ntraces,x,y,GPRfile_new)
    
    # (3.4) Read new data file header and double-check coordinates
       
    gpr_lon, gpr_lat, gpr_mark = npa_read_coord_in_SEGY_PRISM_header(GPRfile_new,ntraces)
        
    istat+=1


 -----------------
  READING FILE: data-OLD/scienzaU4_07.2_CH_2.sgy 
  WRITING FILE: data-NEW-2-constant-spacing/scienzaU4_07.2_CH_2.sgy
  NTRACES in FILE: 382

Encoding Byte Type:  3
Delta (in picos):  49
Nsamples:  1024


     0  LON/LAT:       8.34240000      1.75000000  MARK:    255
     0  LON/LAT:       8.34240000      1.75000000  MARK:    255
   381  LON/LAT:      12.00000000      1.75000000  MARK:      1

 -----------------
  READING FILE: data-OLD/scienzaU4_09.2_CH_2.sgy 
  WRITING FILE: data-NEW-2-constant-spacing/scienzaU4_09.2_CH_2.sgy
  NTRACES in FILE: 303

Encoding Byte Type:  3
Delta (in picos):  49
Nsamples:  1024


     0  LON/LAT:       9.10080000      2.25000000  MARK:    255
     0  LON/LAT:       9.10080000      2.25000000  MARK:    255
   302  LON/LAT:      12.00000000      2.25000000  MARK:      1

 -----------------
  READING FILE: data-OLD/scienzaU4_11.2_CH_2.sgy 
  WRITING FILE: data-NEW-2-constant-spacing/scienzaU4_11.2_CH_2.sgy
  NTRACES in FILE: 284

Enco